In [2]:
import findspark
findspark.init('/home/ubuntu/spark-3.4.1-bin-hadoop3')

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('linearregression').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/03 07:41:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
from pyspark.ml.regression import LinearRegression

In [6]:
training = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

23/12/03 07:41:56 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [7]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [8]:
lr = LinearRegression(featuresCol = 'features', labelCol = 'label', predictionCol='prediction')

In [9]:
lrModel = lr.fit(training)

23/12/03 07:42:04 WARN Instrumentation: [b5e91206] regParam is zero, which might cause numerical instability and overfitting.
23/12/03 07:42:05 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/12/03 07:42:05 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [10]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [11]:
lrModel.intercept

0.14228558260358093

In [12]:
training_summary = lrModel.summary

In [13]:
training_summary.rootMeanSquaredError

10.16309157133015

In [14]:
train_data, test_data = training.randomSplit([0.7,0.3])

In [15]:
train_data.describe()

DataFrame[summary: string, label: string]

In [16]:
model = lr.fit(train_data)

23/12/03 07:42:06 WARN Instrumentation: [5fdd54f5] regParam is zero, which might cause numerical instability and overfitting.


In [17]:
test_results = model.evaluate(test_data)

In [18]:
test_results

In [19]:
test_results.rootMeanSquaredError

10.638454943892762

In [20]:
unlabeled_data = test_data.select('features')

In [21]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [22]:
pred = model.transform(unlabeled_data)

In [23]:
pred.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...|  -3.316548025666691|
|(10,[0,1,2,3,4,5,...| -2.7899738888160637|
|(10,[0,1,2,3,4,5,...|  0.8175428784552493|
|(10,[0,1,2,3,4,5,...|  0.9292922401797732|
|(10,[0,1,2,3,4,5,...|  -1.107959260000301|
|(10,[0,1,2,3,4,5,...| -0.3780150132423192|
|(10,[0,1,2,3,4,5,...|  0.4764922436972179|
|(10,[0,1,2,3,4,5,...| -0.7754530253308057|
|(10,[0,1,2,3,4,5,...|   1.150420006562022|
|(10,[0,1,2,3,4,5,...|  0.7876763336368419|
|(10,[0,1,2,3,4,5,...|   1.443240189773129|
|(10,[0,1,2,3,4,5,...|  -2.655724007341183|
|(10,[0,1,2,3,4,5,...|  0.7613856319996947|
|(10,[0,1,2,3,4,5,...|-0.02392088286354...|
|(10,[0,1,2,3,4,5,...|-0.05870806039218285|
|(10,[0,1,2,3,4,5,...|   2.609293082857177|
|(10,[0,1,2,3,4,5,...| -0.8565051797517543|
|(10,[0,1,2,3,4,5,...|  2.8849280836412445|
|(10,[0,1,2,3,4,5,...|  -1.180059463660931|
|(10,[0,1,2,3,4,5,...| -1.911662